In [2]:
import sys
sys.path.append('./app')  # Add the 'app' directory to the module search path

from engine import get_encoded_skill_columns

skills = get_encoded_skill_columns()
print(skills[:5])


['Active Learning', 'Active Listening', 'Complex Problem Solving', 'Coordination', 'Critical Thinking']


In [3]:
from pathlib import Path

# Use current working directory since __file__ isn't available in notebooks
print((Path.cwd() / "data" / "Skills.xlsx").exists())


True


In [4]:
import pandas as pd
from pathlib import Path

def get_encoded_skill_columns():
    # Load the Excel file
    skills_path = Path.cwd() / "data" / "Skills.xlsx"
    skills_df = pd.read_excel(skills_path)

    # Filter for skills where Scale ID is 'IM' (Importance)
    important_skills_df = skills_df[skills_df["Scale ID"] == "IM"]

    # Get unique skill names from 'Element Name' column
    encoded_columns = sorted(important_skills_df["Element Name"].unique().tolist())

    return encoded_columns

# Now call the function separately
skills = get_encoded_skill_columns()

# Preview first 5 skills
print(skills[:5])


['Active Learning', 'Active Listening', 'Complex Problem Solving', 'Coordination', 'Critical Thinking']


In [5]:
import sys
sys.path.append('./app')  # Add the 'app' directory to the module search path

from engine import get_encoded_skill_columns

skills = get_encoded_skill_columns()
print(skills[:5])

['Active Learning', 'Active Listening', 'Complex Problem Solving', 'Coordination', 'Critical Thinking']


In [6]:
%load_ext autoreload
%autoreload 2


In [7]:
import os

for root, dirs, files in os.walk(".", topdown=True):
    if "engine.py" in files:
        print("Found at:", os.path.join(root, "engine.py"))


Found at: .\app\engine.py


In [12]:
import importlib
import engine
importlib.reload(engine)

from engine import job_profiles_clean, hybrid_similarity_recommender, merge_skills_into_profiles

# Load Skills.xlsx manually
skills_df = pd.read_excel("data/Skills.xlsx")

# Merge them
jobs_df = merge_skills_into_profiles(job_profiles_clean, skills_df)

user_profile = {
    "R": 0.5,
    "I": 0.7,
    "A": 0.2,
    "S": 0.4,
    "E": 0.1,
    "C": 0.6,
    "education_level": "Bachelor's Degree",
    "skills": ["Active Listening", "Critical Thinking"],
    "user_name": "Allan"
}
# Then run recommender
recommendations, debug_info = hybrid_similarity_recommender(user_profile)

                                                 Title  \
369                                       Radiologists   
384                                 Histotechnologists   
175         Medical Scientists, Except Epidemiologists   
385        Medical and Clinical Laboratory Technicians   
207  Environmental Science and Protection Technicia...   
205                             Biological Technicians   
90                   Computer User Support Specialists   
111                         Bioinformatics Technicians   
152    Civil Engineering Technologists and Technicians   
360                                  Anesthesiologists   

     Hybrid Recommendation Score  User RIASEC Similarity  \
369                     0.454521                0.980053   
384                     0.448669                0.965423   
175                     0.448440                0.964850   
385                     0.443949                0.953622   
207                     0.442979                0.951197   
2

In [16]:
[col for col in jobs_df.columns if col.startswith("Skill List_")]

['Skill List_Apprenticeship',
 'Skill List_Job-Related Professional Certification',
 'Skill List_On-Site or In-Plant Training',
 'Skill List_On-the-Job Training',
 'Skill List_Related Work Experience',
 'Skill List_Required Level of Education',
 'Skill List_Active Learning_x',
 'Skill List_Active Listening_x',
 'Skill List_Complex Problem Solving_x',
 'Skill List_Coordination_x',
 'Skill List_Critical Thinking_x',
 'Skill List_Equipment Maintenance_x',
 'Skill List_Equipment Selection_x',
 'Skill List_Installation_x',
 'Skill List_Instructing_x',
 'Skill List_Judgment and Decision Making_x',
 'Skill List_Learning Strategies_x',
 'Skill List_Management of Financial Resources_x',
 'Skill List_Management of Material Resources_x',
 'Skill List_Management of Personnel Resources_x',
 'Skill List_Mathematics_x',
 'Skill List_Monitoring_x',
 'Skill List_Negotiation_x',
 'Skill List_Operation and Control_x',
 'Skill List_Operations Analysis_x',
 'Skill List_Operations Monitoring_x',
 'Skill Lis

In [18]:
user_skills = [skill.lower().strip() for skill in user_profile["skills"]]

job_profile_skills = [col.replace("Skill List_", "").lower().strip() 
                      for col in jobs_df.columns if col.startswith("Skill List_")]

print("User Skills:", user_skills)
print("Job Profile Skills:", job_profile_skills)

User Skills: ['active listening', 'critical thinking']
Job Profile Skills: ['apprenticeship', 'job-related professional certification', 'on-site or in-plant training', 'on-the-job training', 'related work experience', 'required level of education', 'active learning_x', 'active listening_x', 'complex problem solving_x', 'coordination_x', 'critical thinking_x', 'equipment maintenance_x', 'equipment selection_x', 'installation_x', 'instructing_x', 'judgment and decision making_x', 'learning strategies_x', 'management of financial resources_x', 'management of material resources_x', 'management of personnel resources_x', 'mathematics_x', 'monitoring_x', 'negotiation_x', 'operation and control_x', 'operations analysis_x', 'operations monitoring_x', 'persuasion_x', 'programming_x', 'quality control analysis_x', 'reading comprehension_x', 'repairing_x', 'science_x', 'service orientation_x', 'social perceptiveness_x', 'speaking_x', 'systems analysis_x', 'systems evaluation_x', 'technology desig

In [20]:
matched_skills = set(user_skills).intersection(set(job_profile_skills))
print("Matched Skills:", matched_skills)

Matched Skills: {'active listening', 'critical thinking'}


In [24]:
job_profiles_clean = job_profiles_clean.merge(skills_df, on="Title", how="left")

In [26]:
print(job_profiles_clean.columns)
print(skills_df.columns)

Index(['ONET_Code', 'Title', 'Description', 'A', 'C', 'E',
       'First Interest High-Point', 'I', 'R', 'Second Interest High-Point',
       ...
       'Scale Name', 'Data Value_y', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Not Relevant', 'Date',
       'Domain Source'],
      dtype='object', length=116)
Index(['O*NET-SOC Code', 'Title', 'Element ID', 'Element Name', 'Scale ID',
       'Scale Name', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Not Relevant', 'Date',
       'Domain Source'],
      dtype='object')


In [30]:
print("Sample Titles in job_profiles_clean:\n", job_profiles_clean['Title'].unique()[:5])
print("\nSample Titles in skills_df:\n", skills_df['Title'].unique()[:5])


Sample Titles in job_profiles_clean:
 ['Chief Executives' 'Chief Sustainability Officers'
 'General and Operations Managers' 'Advertising and Promotions Managers'
 'Marketing Managers']

Sample Titles in skills_df:
 ['Chief Executives' 'Chief Sustainability Officers'
 'General and Operations Managers' 'Advertising and Promotions Managers'
 'Marketing Managers']


In [32]:
skills_df.rename(columns={"O*NET-SOC Code": "ONET_Code"}, inplace=True)

In [34]:
print(job_profiles_clean.columns)
print(skills_df.columns)

Index(['ONET_Code', 'Title', 'Description', 'A', 'C', 'E',
       'First Interest High-Point', 'I', 'R', 'Second Interest High-Point',
       ...
       'Scale Name', 'Data Value_y', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Not Relevant', 'Date',
       'Domain Source'],
      dtype='object', length=116)
Index(['ONET_Code', 'Title', 'Element ID', 'Element Name', 'Scale ID',
       'Scale Name', 'Data Value', 'N', 'Standard Error', 'Lower CI Bound',
       'Upper CI Bound', 'Recommend Suppress', 'Not Relevant', 'Date',
       'Domain Source'],
      dtype='object')


In [36]:
# Standardize job titles before merging
job_profiles_clean['Title'] = job_profiles_clean['Title'].str.strip().str.lower()
skills_df['Title'] = skills_df['Title'].str.strip().str.lower()

# Merge on standardized title
job_profiles_clean = job_profiles_clean.merge(skills_df, on="Title", how="left")

In [40]:
print(job_profiles_clean.columns)


Index(['ONET_Code_x', 'Title', 'Description', 'A', 'C', 'E',
       'First Interest High-Point', 'I', 'R', 'Second Interest High-Point',
       ...
       'Scale Name_y', 'Data Value', 'N_y', 'Standard Error_y',
       'Lower CI Bound_y', 'Upper CI Bound_y', 'Recommend Suppress_y',
       'Not Relevant_y', 'Date_y', 'Domain Source_y'],
      dtype='object', length=130)


In [50]:
skills_df = skills_df[skills_df['Element Name'].str.contains("skill", case=False, na=False)]

In [52]:
# Keep only necessary columns
skills_df = skills_df[[
    "ONET_Code", "Title", "Element ID", "Element Name", "Scale ID", "Scale Name", "Data Value"
]]

# (Optional) Remove duplicates
skills_df = skills_df.drop_duplicates(subset=["ONET_Code", "Title", "Element ID"])


In [54]:
skills_summary = skills_df.groupby(["ONET_Code", "Title"])["Data Value"].mean().reset_index()
skills_summary.rename(columns={"Data Value": "Avg Skill Value"}, inplace=True)

# Then merge with compressed data
job_profiles_clean = job_profiles_clean.merge(
    skills_summary,
    on=["ONET_Code", "Title"],
    how="left"
)

In [56]:
print("job_profiles_clean rows:", len(job_profiles_clean))
print("skills_df rows:", len(skills_df))
print("Unique job titles in skills_df:", skills_df[['Title', 'ONET_Code']].drop_duplicates().shape[0])

job_profiles_clean rows: 4101300
skills_df rows: 0
Unique job titles in skills_df: 0


In [64]:
skills_df = pd.read_excel("data/Skills.xlsx")  # adjust path if needed
print("Shape after loading:", skills_df.shape)
print("First 5 rows:\n", skills_df.head())

Shape after loading: (61530, 15)
First 5 rows:
   O*NET-SOC Code             Title Element ID           Element Name Scale ID  \
0     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension       IM   
1     11-1011.00  Chief Executives    2.A.1.a  Reading Comprehension       LV   
2     11-1011.00  Chief Executives    2.A.1.b       Active Listening       IM   
3     11-1011.00  Chief Executives    2.A.1.b       Active Listening       LV   
4     11-1011.00  Chief Executives    2.A.1.c                Writing       IM   

   Scale Name  Data Value  N  Standard Error  Lower CI Bound  Upper CI Bound  \
0  Importance        4.12  8          0.1250          3.8800          4.3700   
1       Level        4.62  8          0.1830          4.2664          4.9836   
2  Importance        4.00  8          0.0000          4.0000          4.0000   
3       Level        4.75  8          0.1637          4.4292          5.0708   
4  Importance        4.12  8          0.1250          3.8800     

In [66]:
# Step 1: Filter only "Importance" scale rows
important_skills_df = skills_df[skills_df['Scale Name'] == 'Importance']

# Step 2: Pivot so each skill is a column
pivoted_skills = important_skills_df.pivot_table(
    index='O*NET-SOC Code',
    columns='Element Name',
    values='Data Value'
)

# Step 3: Reset index and inspect
pivoted_skills.reset_index(inplace=True)
print("Shape after pivoting:", pivoted_skills.shape)
print("Sample rows:\n", pivoted_skills.head())

Shape after pivoting: (879, 36)
Sample rows:
 Element Name O*NET-SOC Code  Active Learning  Active Listening  \
0                11-1011.00             3.75              4.00   
1                11-1011.03             3.75              4.00   
2                11-1021.00             3.62              4.00   
3                11-2011.00             3.25              4.12   
4                11-2021.00             3.88              3.88   

Element Name  Complex Problem Solving  Coordination  Critical Thinking  \
0                                4.38          4.25               4.38   
1                                4.00          3.75               4.12   
2                                3.62          3.88               3.88   
3                                3.50          3.50               4.00   
4                                3.62          3.50               3.88   

Element Name  Equipment Maintenance  Equipment Selection  Installation  \
0                               1.0   

In [68]:
pivoted_skills.rename(columns={'O*NET-SOC Code': 'ONET_Code'}, inplace=True)

In [70]:
job_profiles_with_skills = pd.merge(job_profiles_clean, pivoted_skills, on='ONET_Code', how='left')

In [76]:
import numpy as np

# Sample user input (you can customize this)
user_input = {
    'Critical Thinking': 4.5,
    'Speaking': 4.0,
    'Coordination': 3.5,
    'Time Management': 4.0,
    'Judgment and Decision Making': 4.2,
    'Reading Comprehension': 4.3,
    'Monitoring': 3.9,
    'Systems Analysis': 4.1,
    'Writing': 3.8,
    'Active Listening': 4.0
}

skill_columns = pivoted_skills.columns.tolist()[1:]  # drop 'ONET_Code'

# Create the user skill vector with zeros for missing inputs
user_skills_vector = np.array([user_input.get(skill, 0.0) for skill in skill_columns])

In [78]:
print("User vector length:", len(user_skills_vector))
print("Job vector length:", len(job_profiles_with_skills[skill_columns].iloc[0].values))


User vector length: 35
Job vector length: 35


In [80]:
from sklearn.metrics.pairwise import cosine_similarity

job_vectors = job_profiles_with_skills[skill_columns].fillna(0).values
similarities = cosine_similarity([user_skills_vector], job_vectors)[0]

job_profiles_with_skills['User Skill Similarity'] = similarities

In [82]:
top_matches = job_profiles_with_skills.sort_values(by='User Skill Similarity', ascending=False).head(10)
display(top_matches[['Title', 'User Skill Similarity']])

,Title,User Skill Similarity
2773411,proofreaders and copy markers,0.751481
2775040,proofreaders and copy markers,0.751481
2775033,proofreaders and copy markers,0.751481
2775034,proofreaders and copy markers,0.751481
2775035,proofreaders and copy markers,0.751481
2775036,proofreaders and copy markers,0.751481
2775037,proofreaders and copy markers,0.751481
2775038,proofreaders and copy markers,0.751481
2775039,proofreaders and copy markers,0.751481
2775041,proofreaders and copy markers,0.751481


In [84]:
top_matches_unique = job_profiles_with_skills.sort_values(
    by='User Skill Similarity', ascending=False
).drop_duplicates(subset='Title').head(10)

display(top_matches_unique[['Title', 'User Skill Similarity']])

,Title,User Skill Similarity
2773411,proofreaders and copy markers,0.751481
2064529,medical transcriptionists,0.746481
1567994,technical writers,0.733998
2735429,legal secretaries and administrative assistants,0.726979
1153137,"title examiners, abstractors, and searchers",0.724880
1143337,"judges, magistrate judges, and magistrates",0.724047
1569632,writers and authors,0.723544
1146602,paralegals and legal assistants,0.720114
1582703,court reporters and simultaneous captioners,0.719529
1574536,"poets, lyricists and creative writers",0.717836
